In [1]:
import pandas as pd
import numpy as np

/Users/giorgia/.pyenv/versions/3.7.8/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
def get_daily_cases(data, col_name='series_1'):
    series = data.copy()
    series = series.groupby(by='Date_statistics').count()
    series = series.rename(columns={'Date_file': col_name})
    return series[col_name]


def make_measures_0_1(column):
    column=np.where(column==daily_data[column.name].max(),column, 0)
    column=np.where(column==0,column, 1)
    return column

In [10]:
#Load Data
measures = pd.read_excel('measures_strictness.xlsx', engine='openpyxl').rename(columns={'Unnamed: 0':'Date'})
cases_national= pd.read_csv('Data/COVID-19_casus_landelijk.csv', sep=';')
behaviour_data= pd.read_csv('Data/COVID-19_gedrag.csv', sep=';')

In [11]:
# Create datasets

# Put date in the right format, merge with measures data
cases_national['Date_statistics'] = pd.to_datetime(cases_national['Date_statistics'],errors='ignore', dayfirst=True)
daily_cases = get_daily_cases(cases_national, 'Number of new cases').to_frame().reset_index()
daily_cases['Date_statistics'] = pd.to_datetime(daily_cases['Date_statistics'])
daily_data = daily_cases.merge(measures, left_on='Date_statistics', right_on='Date', how='left').drop(['Date', 'stringency_index', 'government_response_index', 'containment_health_index', 'economic_support_index'], axis=1)

# Put date in the right format, filter to simplify the dataset, drop useless columns
behaviour_data['Date_of_measurement'] = pd.to_datetime(behaviour_data['Date_of_measurement'],errors='ignore')
behaviour_data = behaviour_data.loc[lambda d: (d.Region_name=='Nederland') & (d.Subgroup == 'Totaal') & (d.Sample_size != 0) & ((d.Indicator_category.str.contains('Naleving')) | (d.Indicator_category.str.contains('Zorgen')))]
behaviour_data= behaviour_data.drop(['Date_of_report', 'Wave','Region_code', 'Subgroup', 'Figure_type', 'Region_name', 'Subgroup_category', 'Change_wrt_previous_measurement'], axis=1)
behaviour_data['Normalised_Value'] = behaviour_data['Value'] / behaviour_data['Sample_size']

In [12]:
new=daily_data.drop(['Date_statistics','Number of new cases'],axis=1).apply(make_measures_0_1,axis=0,result_type='expand')
daily_data=pd.concat([daily_data['Date_statistics'],daily_data['Number of new cases'],new],axis=1)
daily_data.to_csv('daily_data.csv',index=False)
behaviour_data.to_csv('behaviour_data.csv',index=False)

In [13]:
behaviour_data

,Date_of_measurement,Indicator_category,Indicator,Sample_size,Value,Lower_limit,Upper_limit,Normalised_Value
1656,2020-09-28,Naleving,Bij_klachten_blijf_thuis,929,46.1,42.5,49.7,0.049623
1692,2020-09-28,Naleving,Bij_klachten_laat_testen,1012,32.2,29.1,35.4,0.031818
1836,2020-09-28,Naleving,Houd_1_5m_afstand,4966,61.6,61.1,62.2,0.012404
1872,2020-09-28,Naleving,Ontvang_max_bezoekers_thuis,5025,97.3,96.7,97.7,0.019363
1944,2020-09-28,Naleving,Vermijd_drukke_plekken,5025,63.3,61.8,64.7,0.012597
...,...,...,...,...,...,...,...,...
19764,2021-03-08,Naleving,Thuisgewerkte_uren,1763,64.0,62.0,66.0,0.036302
19800,2021-03-08,Naleving,Vermijd_drukke_plekken,5076,74.1,72.8,75.4,0.014598
19836,2021-03-08,Naleving,Was_vaak_je_handen,5076,33.8,32.4,35.2,0.006659
19872,2021-03-08,Naleving,Werkt_thuis,3149,67.4,65.6,69.2,0.021404


In [20]:
daily_data

,Date_statistics,Number of new cases,c1_school_closing,c2_workplace_closing,c3_cancel_public_events,c4_restrictions_on_gatherings,c5_close_public_transport,c6_stay_at_home_requirements,c7_movement_restriction,c8_international_travel,h1_public_information_campaigns,h2_testing_policy,h3_contact_tracing,h6_facial_coverings,h7_vaccination_policy,h8_protection_of_elderly_people
0,2020-01-01,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2020-01-04,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2020-01-06,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2020-01-16,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2020-01-20,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,2021-02-12,3459,0,1,1,1,1,1,1,1,1,1,0,1,1,0
385,2021-02-13,3370,0,1,1,1,1,1,1,1,1,1,0,1,1,0
386,2021-02-14,3490,0,1,1,1,1,1,1,1,1,1,0,1,1,0
387,2021-02-15,4058,0,1,1,1,1,1,1,1,1,1,0,1,1,0
